In [1]:
import numpy as np
import pandas as pd
import re
from numpy.random import default_rng
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns',100) 
pd.set_option('max_rows',500)  
File_Path = 'DataAnalyst.csv'

In [2]:
df = pd.read_csv(File_Path)

## Clean Salary Estimate

In [3]:
df['Lo Salary'] = df['Salary Estimate'].str.extract(r'^\s*\$(\d+\.*\d*[kK])[^0-9]', expand=True)
df['Lo Salary'] = df['Lo Salary'].str.replace(r'[kK]$', '000')
df['Hi Salary'] = df['Salary Estimate'].str.extract(r'\s*-\s*\$(\d+\.*\d*[kK])[^0-9]', expand=True)
df['Hi Salary'] = df['Hi Salary'].str.replace(r'[kK]$', '000')
df.drop(['Salary Estimate'], axis=1, inplace=True)

## Clean Company Name

In [4]:
df['Company Name'] = df['Company Name'].str.replace('\\n\d\.*\d*','')

## Clean Size

In [5]:
# constraint, number employees should be at least 1
# if less than 1, set to 'Unknown'
def RemoveStranger(x):
    x = x.strip()
    if re.match('(^-[1-9]+\d*$)|(^0$)', x):
        return 'Unkonwn'
    return x

df['Size'] = df['Size'].apply(RemoveStranger)
df['Size'] = df['Size'].str.replace(r' employees$', '')
df['Size'] = df['Size'].str.replace(r'\s*to\s*', ',')
df[['Lo Em','Hi Em']] = df['Size'].str.split(',', n=1, expand=True)
df['Lo Em'] = df['Lo Em'].str.replace('+', '')
df['Hi Em'] = df['Hi Em'].astype(str)
df['Hi Em'] = df['Hi Em'].str.replace('None', 'Unknown')
df = df.drop(['Size'],axis=1)

## Clean Headquarters

In [6]:
df.Headquarters = df.Headquarters.apply(RemoveStranger)

## Clean Industry

In [7]:
df.Industry = df.Industry.apply(RemoveStranger)

## Clean Sector

In [8]:
df.Sector = df.Sector.apply(RemoveStranger)

## Clean Revenue

In [9]:
r_dic = {'million': 1000000, 'billion':1000000000}

In [10]:
df.Revenue = df.Revenue.apply(RemoveStranger)

In [11]:
df.Revenue.astype('str')
def preliminary_clean(x):
    x = x.strip()
#     pat = r'^(Less than)*[^0-9]*(\d*)(\+)*[^0-9a-z]*(million|billion)*[^0-9]*(\d+)*[^0-9a-z]*(million|billion)'
    pat = r'^(Less than)*[^0-9]*(\d*)[^0-9a-z]*(million|billion)*[^0-9]*(\d+)*[^0-9a-z]*(million|billion)'
    m = re.match(pat, x, re.I)
    res = ''
    if m:
        if m.group(1):
            res += str(m.group(1))
        if m.group(2):
            lo_r = int(m.group(2))
        if m.group(3):
            lo_r *= int(r_dic[str(m.group(3))])
            res += str(lo_r)
        if m.group(3) == None:
            lo_r *= int(r_dic[str(m.group(5))])
            res += str(lo_r)            
#         if m.group(3):
#             res += str(m.group(3))
        if m.group(4):
            res+=','
            hi_r = int(m.group(4))
            hi_r *= int(r_dic[str(m.group(5))])
            res += str(hi_r)
    else:
        res = 'Unknown'
    return res

df.Revenue = df.Revenue.apply(preliminary_clean)

In [12]:
df[['Lo Rev', 'Hi Rev']] = df.Revenue.str.split(',', expand=True)

df['Hi Rev'] = df['Hi Rev'].astype(str)
for i, v in enumerate (df['Hi Rev'].values):
    v = v.strip()
    if v == 'None':
        if 'Less than' in df['Lo Rev'][i]:
            v = df['Lo Rev'][i].strip('Less than')
            df.loc[i,'Lo Rev'] = 'Unknown'
        else:
            v = 'Unknown'
    df.loc[i,'Hi Rev'] = v       

In [13]:
# to do: 公司分级； JD关键词